In [1]:
# coding: utf-8

In [2]:
import time 
import json
import requests
import pandas as pd
import pickle
import matplotlib.pyplot as plt 
from calendar import monthrange
from datetime import datetime


pd.options.mode.chained_assignment = None  # default='warn'

In [3]:
# choix du mois que l'on veut récupérer par API 
year = int(input("pour quelle année ? (format YYYY)?"))
month = int(input("pour quel mois (de 1 à 12) ?")) # sert ensuite pour générer le bon lien à requeter.

In [4]:
# Obligation de faire des requêtes journalières car l'API renvoie du json "cassé" pour certains jours et produit ainsi une 
# erreur réponse 502 à la requête globale. 
# L'API est en heure française, pas de soucis de gestion d'heure UTC

link_part1 = "https://download.data.grandlyon.com/ws/timeseries/jcd_jcdecaux.historiquevelov/all.json?horodate__gte="
link_part2 = "&maxfeatures=600000"

for i in range(1, (monthrange(year,month)[1])+1) : 
    date_min = str(year) + "-" + str(month) + "-" + str(i)
    
    # gestion de la récupération des jours différente pour le dernier jour du mois
    if i == (monthrange(year,month)[1]) : 
        # changement de mois pour la date max
        date_max = str(year) + "-" + str(month+1) + "-1" 
    else : 
        date_max = str(year) + "-" + str(month) + "-" + str(i+1)
    
    lien_entier = link_part1 + date_min + "&horodate__lte=" + date_max + link_part2
    response = requests.get(lien_entier)
    
    try : 
        data_endpoint = json.loads(response.text)
        
        # création d'un df journalier : 
        historique_journalier = pd.json_normalize(data_endpoint, record_path="values")
        
        # allegement du df : choix de conserver uniquement ces colonnes : 
        historique_journalier=historique_journalier[["horodate",
                                           "number", 
                                           "status",
                                           'total_stands.availabilities.bikes', 
                                           'total_stands.availabilities.electricalBikes',
                                           'total_stands.availabilities.stands',
                                           'total_stands.capacity']] 
        
        # concaténation dans un df unique mensuel
        if i == 1 : #initialisation du df
            historique = historique_journalier
        else : 
            historique = pd.concat([historique,historique_journalier])
            
    except KeyError as e : 
        print(e)
    except json.decoder.JSONDecodeError as e :    
        print(e)

"None of [Index(['horodate', 'number', 'status', 'total_stands.availabilities.bikes',\n       'total_stands.availabilities.electricalBikes',\n       'total_stands.availabilities.stands', 'total_stands.capacity'],\n      dtype='object')] are in the [columns]"


In [5]:
historique

,horodate,number,status,total_stands.availabilities.bikes,total_stands.availabilities.electricalBikes,total_stands.availabilities.stands,total_stands.capacity
0,2023-02-01 23:52:08+01:00,1,CLOSED,0,0,99,103
1,2023-02-01 23:42:08+01:00,1,CLOSED,0,0,99,103
2,2023-02-01 23:32:08+01:00,1,CLOSED,0,0,99,103
3,2023-02-01 23:22:08+01:00,1,CLOSED,0,0,99,103
4,2023-02-01 23:12:08+01:00,1,CLOSED,0,0,99,103
...,...,...,...,...,...,...,...
97917,2023-02-28 00:34:31+01:00,34002,OPEN,8,5,7,15
97918,2023-02-28 00:28:31+01:00,34002,OPEN,8,5,7,15
97919,2023-02-28 00:18:26+01:00,34002,OPEN,8,5,7,15
97920,2023-02-28 00:15:33+01:00,34002,OPEN,8,5,7,15


In [6]:
historique.value_counts("status")

status
OPEN      2340856
CLOSED      10972
dtype: int64

In [7]:
historique.value_counts("number") # nombre de relevé par station

number
3011     12268
1002     11912
10006    10870
3019     10503
1022     10227
         ...  
34002     3650
19001     3637
5044      3554
201       3063
1016      2330
Length: 432, dtype: int64

In [8]:
historique.rename(columns={'total_stands.availabilities.bikes' : 'availabilities.all.types',
            'total_stands.availabilities.electricalBikes' : 'availabilities.electricalBikes',
            'total_stands.availabilities.stands' : "availabilities.stands", 
            'total_stands.capacity' :"capacity"}, inplace=True)

historique.drop(historique[historique["status"]=="CLOSED"].index,inplace=True)
historique.drop_duplicates(keep="first", inplace=True)

In [10]:
historique.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2170162 entries, 188 to 97921
Data columns (total 7 columns):
 #   Column                          Dtype 
---  ------                          ----- 
 0   horodate                        object
 1   number                          int64 
 2   status                          object
 3   availabilities.all.types        int64 
 4   availabilities.electricalBikes  int64 
 5   availabilities.stands           int64 
 6   capacity                        int64 
dtypes: int64(5), object(2)
memory usage: 132.5+ MB


In [13]:
# Allegement du df à un relevé toutes les 15 min par station 

# récupération des numéros de stations pour classer les horaires par station
liste_numeros_stations = list(historique["number"].unique())

# passage au format datetime pour pouvoir utiliser .asfreq
historique['horodate'] = pd.to_datetime(historique['horodate'], format='%Y-%m-%d %H:%M:%S%Z')
historique.sort_values(by=["number", "horodate"], inplace=True)

# les relevés sont regroupés par station 
for i in liste_numeros_stations :
    station = historique.loc[historique["number"] == i]
    
    # je veux que mes horaires soient homogènes sur chaque station (le premier relevé de la 
    # journée n'est pas à la même minute pour toutes les stations)
    
    # je copie la ligne avec la date la plus petite puis je modifie son horaire
    heure_initialisation_asfreq15 = station[station["horodate"] == station.min()[0]].squeeze()
    heure_initialisation_asfreq15["horodate"] = heure_initialisation_asfreq15["horodate"].replace(hour=00, minute=00, second=0)
    
    station = pd.concat([station, heure_initialisation_asfreq15.to_frame().T], axis=0)
    station.sort_values(by="horodate", inplace=True)
    station.drop_duplicates(keep="last", inplace=True)
    station.set_index("horodate", inplace=True) 
    station = station.asfreq('15T', method='bfill') 
    station.reset_index(drop=False, inplace=True)
    
    # pour la premiere station traitée je crée un nouveau df, pour les stations suivante je concatène avec le df crée en 0
    if i == liste_numeros_stations[0] : 
        historique15 = station
    else : 
        historique15 = pd.concat([historique15,station])
        
       
        
# stokage du df en format pickle
nom_fichier = "dump" + str(year) + "_" + str(month) + "_15min"
with open(nom_fichier + ".pkl", 'wb') as f: # nom du fichier à créer
    pickle.dump(historique15, f)


In [14]:
historique15

,horodate,number,status,availabilities.all.types,availabilities.electricalBikes,availabilities.stands,capacity
0,2023-02-01 00:00:00+01:00,555,OPEN,0,0,3,5
1,2023-02-01 00:15:00+01:00,555,OPEN,0,0,3,5
2,2023-02-01 00:30:00+01:00,555,OPEN,0,0,3,5
3,2023-02-01 00:45:00+01:00,555,OPEN,0,0,3,5
4,2023-02-01 01:00:00+01:00,555,OPEN,0,0,3,5
...,...,...,...,...,...,...,...
98,2023-02-26 00:30:00+01:00,1,OPEN,35,3,68,103
99,2023-02-26 00:45:00+01:00,1,OPEN,35,3,68,103
100,2023-02-26 01:00:00+01:00,1,OPEN,35,3,68,103
101,2023-02-26 01:15:00+01:00,1,OPEN,35,3,68,103


In [15]:
historique15["number"].value_counts()

6005    2689
7002    2689
555     2688
9006    2688
9029    2688
        ... 
5007    2544
5047    2496
5050    1964
4008    1404
1        103
Name: number, Length: 431, dtype: int64